<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/ask_0_mixed_pubmed_bigtweet_ask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'AskAPatient.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-0.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heel pain
2,4,purple sploches around waist
3,7,goiter
4,8,ARTHRITIS


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-0.test.csv",header=None)
test.head()

,0,1
0,0,'scared' feeling
1,2,heel pain
2,3,cracking sensations in my joints
3,5,fast and slow heartbeat
4,5,increased heart rate


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'mixed_pubmed_bigtweet_askpatient.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'of',
 'the',
 'in',
 'lipitor',
 'and',
 'to',
 'a',
 'muscle',
 'with',
 'for',
 'was',
 'diclofenac',
 'were',
 'severe',
 'on',
 'my',
 'is',
 'myalgia',
 'lower',
 'fatigue',
 'rt',
 'loss',
 'by',
 'limb',
 'cramp',
 'sodium',
 'weakness',
 'at',
 'disease',
 'that',
 'stomach',
 'or',
 'as',
 'back',
 'depression',
 '-',
 'patients',
 'from',
 'i',
 'cramps',
 'this',
 'not',
 'upper',
 'gas',
 'joint',
 'drug',
 'memory',
 'leg',
 'you',
 'mg',
 'all',
 'legs',
 'pains',
 'knee',
 'headache',
 'be',
 'group',
 'are',
 'study',
 'shoulder',
 'unable',
 'arthralgia',
 'gastrointestinal',
 'treatment',
 'excessive',
 'foot',
 'neck',
 'after',
 'it',
 'arthritis',
 'an',
 'aches',
 'have',
 'hip',
 'amp',
 'voltaren',
 'like',
 'p',
 'numbness',
 'insomnia',
 'release',
 'increased',
 'arthrotec',
 'no',
 '/',
 'nausea',
 'both',
 'time',
 'bleeding',
 'gluten',
 'abdominal',
 'tire

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.863773,3.484704,0.443692,1:28:17


In [22]:
# learn.fit_one_cycle(1, 1e-2)

In [23]:
# learn.fit_one_cycle(1, 1e-3)

In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.516819,3.422022,0.449406,1:28:44


In [25]:
learn.save_encoder('pubmed_cadec_askpatient_mixed_fold0_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_cadec_askpatient_mixed_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.860976,3.672613,0.372781,03:09


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.846909,3.036216,0.468639,03:03


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.249826,2.602762,0.566864,07:07


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.950286,2.300060,0.639053,07:54
1,1.684836,2.157590,0.640237,07:11


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.466461,1.929238,0.679290,07:46
1,1.316434,1.831546,0.693491,08:09


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.184552,1.725348,0.706509,07:25
1,1.050836,1.630139,0.717160,07:45


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.954527,1.598826,0.730178,07:46
1,0.854609,1.543065,0.733728,06:45


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.815929,1.465783,0.747929,07:30
1,0.673965,1.445366,0.749112,07:17


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.662553,1.426980,0.762130,07:45
1,0.627913,1.352136,0.768047,07:40
2,0.540390,1.327803,0.773965,07:41
3,0.473002,1.306731,0.779882,07:44


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.459134,1.335413,0.781065,07:40
1,0.512938,1.292035,0.785799,07:27
2,0.406144,1.314688,0.786982,07:17
3,0.361674,1.296540,0.785799,06:47


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.365914,1.369171,0.786982,07:09
1,0.396390,1.333472,0.784615,07:15
2,0.363634,1.304574,0.800000,07:26
3,0.287046,1.287530,0.800000,06:59


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.284048,1.401693,0.786982,07:14
1,0.341658,1.414474,0.786982,07:01
2,0.312788,1.357199,0.803550,07:26
3,0.253231,1.314022,0.798817,07:34


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.253297,1.356772,0.794083,07:10
1,0.289727,1.394077,0.794083,07:33
2,0.256374,1.321007,0.798817,07:36
3,0.204630,1.324804,0.802367,07:31


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.215773,1.339109,0.798817,07:28
1,0.257835,1.370425,0.808284,07:43
2,0.234104,1.346141,0.811834,06:53
3,0.206337,1.446292,0.808284,06:49


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.184634,1.429772,0.802367,07:45
1,0.227864,1.399205,0.801183,07:48
2,0.224387,1.325884,0.811834,07:34
3,0.177355,1.348577,0.808284,07:43


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.212445,1.429199,0.797633,07:18
1,0.205844,1.380416,0.811834,07:30


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.200660,1.446141,0.804734,07:26
1,0.188579,1.432647,0.804734,07:04


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.177397,1.428565,0.802367,06:48


In [44]:
# save the best model

learn.save_encoder('pubmed_cadec_askpatient_mixed_fold0')

# Part three: Predict on the test dataset

In [45]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

867
867
[0, 2, 509, 561, 5, 8, 8, 8, 8, 8, 8, 8, 891, 5, 21, 22, 325, 27, 27, 644, 29, 289, 30, 30, 31, 31, 32, 34, 427, 37, 37, 83, 37, 37, 37, 37, 983, 44, 439, 49, 49, 325, 325, 49, 49, 49, 49, 49, 49, 100, 100, 100, 100, 100, 53, 53, 58, 61, 71, 71, 491, 76, 76, 79, 79, 79, 79, 79, 574, 949, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 111, 228, 79, 79, 80, 80, 80, 80, 80, 80, 80, 80, 80, 88, 89, 627, 137, 90, 91, 92, 94, 94, 94, 94, 94, 94, 94, 94, 96, 337, 98, 683, 98, 100, 360, 100, 100, 100, 100, 100, 100, 100, 100, 808, 337, 108, 108, 108, 209, 873, 111, 111, 111, 4, 111, 190, 117, 120, 402, 402, 122, 125, 1025, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 127, 129, 131, 745, 142, 142, 144, 344, 301, 151, 151, 79, 144, 157, 158, 165, 165, 166, 166, 166, 291, 172, 172, 172, 172, 172, 172, 172, 79, 524, 182, 182, 182, 182, 182, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

632
0.7289504036908881
